##### 版權 2024 Google LLC.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: 使用 Python 呼叫函式


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/function_calling_python_quickstart"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上檢視</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/function_calling_python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/function_calling_python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視來源</a>
  </td>
</table>


你可以向 Gemini 模型提供功能描述。該模型可能會要求你呼叫某個功能，並返回結果以幫助該模型處理你的查詢。


## 設定


### 安裝 Python SDK

Gemini API 的 Python SDK 包含在 [`google-generativeai`](https://pypi.org/project/google-generativeai/) 套件中。使用 pip 安裝相依性：


In [ ]:
!pip install -U -q google-generativeai

### 匯入套件


匯入必要的套件。


In [3]:
import pathlib
import textwrap
import time

import google.generativeai as genai


from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### 設定你的 API 金鑰

在開始使用 Gemini API 前，你必須先取得 API 金鑰。如果你還沒有，請在 Google AI Studio 點一下建立金鑰。

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">取得 API 金鑰</a>


在 Colab 中，在左側面板中的「🔑」下方新增金鑰至秘鑰管理員。請將其命名為 `API_KEY`。


一旦你取得了 API 金鑰，請將其傳遞到 SDK。你可以用兩種方式進行此事：

* 將金鑰放入 `GOOGLE_API_KEY` 環境變數中 (SDK 會自動從中撷取)。
* 將金鑰傳遞到 `genai.configure(api_key=...)`


In [22]:
try:
    # Used to securely store your API key
    from google.colab import userdata
    
    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
    
genai.configure(api_key=GOOGLE_API_KEY)

## 函式基礎


建立 `genai.GenerativeModel` 時，你可以將函式清單傳遞至 `tools` 參數。

> 重要：SDK 會將函式引數的類型註解轉換成 API 所了解的格式。API 僅支援有限選擇的引數類型，而此次自動轉換僅支援其中一部分：`int | float | bool | str | list | dict`


In [5]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[multiply])

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x10e73fe90>,
)

推薦使用聊天介面來呼叫函式的方法。主要原因是 `FunctionCalls` 很適合聊天中的多輪對話結構。


In [6]:
chat = model.start_chat(enable_automatic_function_calling=True)

開放自動功能呼叫，如果模型要求 `chat.send_message` 會自動呼叫你的功能。

它似乎只需回傳封裝正確答案的文字回應：


In [7]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'The total number of mittens is 2508.'

In [8]:
57*44

2508

如果你查看 `ChatSession.history`，便可以看見以下順序的事件：

1. 你發送問題。
2. 模型回應 `glm.FunctionCall`。
3. `genai.ChatSession` 在本機執行功能，並送回 `glm.FunctionResponse` 給模型。
4. 模型在它的回答中使用功能輸出。


In [9]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}
--------------------------------------------------------------------------------
model -> {'text': 'The total number of mittens is 2508.'}
--------------------------------------------------------------------------------


一般來說，狀態圖是：

<img src="https://ai.google.dev/images/tutorials/function_call_state_diagram.png" alt="模型始終可以回覆文字或 FunctionCall，如果模型發送 FunctionCall，用戶必須回覆 FunctionResponse" width=50%>


此模型可以在傳回文字回應之前以多個函式呼叫來回應，而函式呼叫出現在文字回應之前。


雖然這一切皆自動處理，倘若你需要更自訂的控制，你可以：

- 保留預設值 `enable_automatic_function_calling=False` 和自行處理 `glm.FunctionCall` 回應。
- 或使用 `GenerativeModel.generate_content`，在此你也需要管理對話紀錄。


## [可選] 低等存取


從 python函式自動擷取結構在某些情況下無法運作。例如：它無法處理描述巢狀的字典物件，但 API 支援這項功能。API 能描述任何下列類型：

```
AllowType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType]
```

`google.ai.generativelanguage` 函式庫提供對低階類型的存取，讓你完全掌控。


In [10]:
import google.ai.generativelanguage as glm

首先查看模型的 `_tools` 屬性，你可以看到，它說明了你傳遞給模型的功能：


In [11]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                             tools=[multiply])

model._tools.to_proto()

[function_declarations {
   name: "multiply"
   description: "returns a * b."
   parameters {
     type_: OBJECT
     properties {
       key: "b"
       value {
         type_: NUMBER
       }
     }
     properties {
       key: "a"
       value {
         type_: NUMBER
       }
     }
     required: "a"
     required: "b"
   }
 }]

這會傳回傳送至 API 的 `glm.Tool` 物件清單。如果列印格式不熟悉，那是因為這些是 Google protobuf 類。每個 `glm.Tool` (本例中為 1) 含有 `glm.FunctionDeclarations` 清單，用以描述函式及其引數。


這是一個使用 `glm` 類封裝的相同乘法函式的宣告。

請注意，這些類僅描述 API 的函式，不包含函式的實作。所以使用這個方式並不適用於自動函式呼叫，但函式並不需要總是有一個實作。


In [12]:
calculator = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='multiply',
        description="Returns the product of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a':glm.Schema(type=glm.Type.NUMBER),
                'b':glm.Schema(type=glm.Type.NUMBER)
            },
            required=['a','b']
        )
      )
    ])

等價地，你可以將此描述為一個與 JSON 相容的物件：


In [13]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': 'Returns the product of two numbers.',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'}},
       'required': ['a', 'b']}}]}

In [14]:
glm.Tool(calculator)

function_declarations {
  name: "multiply"
  description: "Returns the product of two numbers."
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
      }
    }
    required: "a"
    required: "b"
  }
}

無論如何，你可以將 `glm.Tool` 或工具清單的表示遞傳給


In [15]:
model = genai.GenerativeModel('gemini-pro', tools=calculator)
chat = model.start_chat()

response = chat.send_message(
    f"What's 234551 X 325552 ?",
)

如同之前模型回傳 `glm.FunctionCall` 呼叫計算器的 `multiply` 函式:


In [16]:
response.candidates

[index: 0
content {
  parts {
    function_call {
      name: "multiply"
      args {
        fields {
          key: "b"
          value {
            number_value: 325552
          }
        }
        fields {
          key: "a"
          value {
            number_value: 234551
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
]

親自執行函式：


In [17]:
fc = response.candidates[0].content.parts[0].function_call
assert fc.name == 'multiply'

result = fc.args['a'] * fc.args['b']
result

76358547152.0

將結果傳送給模型，繼續對話：


In [18]:
response = chat.send_message(
    glm.Content(
    parts=[glm.Part(
        function_response = glm.FunctionResponse(
          name='multiply',
          response={'result': result}))]))

## 總結

基本函式呼叫在 SDK 中提供支援。請記得由於自然的前後架構，因此使用 chat 模式比較容易管理。你負責實際呼叫函式並將結果傳回模型，以便它能產生文字回應。
